네가지 모델 rmse 가장 좋은 모델 선택
- -> 이 모델이 좋다
- -> 그 모델을 기준으로 위험도 지표 산출
- -> 4가지 모델 살펴보고 그 중 겹치는 것... 
- -> 몇개 맞췄는지....
실제로 맞춘 기준

만약 상위 10개 상위 20개 중 모델 비교

가장 좋은 모델을 더 반영해주기 

모델 보면 knn feature importance + - 나오지 않지만
regression은 긍정/부정적 지표 알 수 있다. 

합쳐서 위험도지표 나온다. 
위험도지표 계산 해서 안전, 위험 한 지역 있다 
-> 상위 20개 위험지역 : 해결책 제시 

--> 위험지역... 선정


In [3]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd

from shapely.geometry import Point, shape, mapping
from shapely.ops import unary_union

import matplotlib.pyplot as plt

from geoband.API import *
import folium
import json
import geopandas as gpd
from sklearn.metrics import mean_squared_log_error

In [4]:
pd.read_csv("dataset2.txt", delimiter = "\t",encoding="euc_kr")

,Unnamed: 0,gid,accident_cnt,geometry,schoolzone_num,safetysign_num,bump_num,cctv_num,child_num,school_num,...,sidewalk_num,ill_park,car_num,node,park_YN,foot_traf_mean,traffic,bigcar_ratio,rushhour_freq,road_width
0,3,다사564096,1,MULTIPOLYGON (((127.0087950432604 37.184159165...,0,1,0,1,0.0,0,...,0,0,0,1,0,42.714375,4779.710000,0.232100,4.6300,0.500
1,11,다사569091,0,MULTIPOLYGON (((127.0144567913441 37.179675470...,0,0,0,0,0.0,0,...,0,0,0,1,0,17.366065,2982.066667,0.338792,35.0050,0.500
2,12,다사569092,0,MULTIPOLYGON (((127.0144510220025 37.180576853...,0,1,0,0,0.0,0,...,0,0,1,0,1,27.409931,4835.990000,0.232096,5.2850,0.500
3,13,다사569093,0,MULTIPOLYGON (((127.014445252404 37.1814782360...,0,0,0,0,0.0,0,...,0,0,0,0,1,7.110000,4835.990000,0.232096,5.2850,0.500
4,14,다사570088,0,MULTIPOLYGON (((127.0156005712868 37.176975930...,0,0,0,0,0.0,0,...,0,0,3,0,0,11.820111,1607.680000,0.372530,99.7800,0.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,1704,다사640065,0,MULTIPOLYGON (((127.0945650795965 37.156539968...,0,0,0,0,0.0,0,...,2,5,0,0,0,2.442500,1936.405000,0.117912,29.0625,0.625
1588,1705,다사640066,0,MULTIPOLYGON (((127.0945602661362 37.157441364...,0,0,0,0,0.0,0,...,2,77,0,0,0,171.749722,2293.930000,0.057351,19.6000,0.500
1589,1706,다사640067,0,MULTIPOLYGON (((127.0945554524616 37.158342760...,0,0,0,0,0.0,0,...,2,4,0,0,0,0.000000,2293.930000,0.057351,19.6000,0.500
1590,1707,다사641063,0,MULTIPOLYGON (((127.0957008616508 37.154741021...,0,0,0,0,0.0,0,...,2,9,0,1,0,9.059167,1578.880000,0.178473,38.5250,0.750


In [6]:
dataset = pd.read_csv("dataset2.txt", delimiter = "\t",encoding="euc_kr")
dataset = dataset.drop(columns=['Unnamed: 0'])
dataset.head()

,gid,accident_cnt,geometry,schoolzone_num,safetysign_num,bump_num,cctv_num,child_num,school_num,facility_num,...,sidewalk_num,ill_park,car_num,node,park_YN,foot_traf_mean,traffic,bigcar_ratio,rushhour_freq,road_width
0,다사564096,1,MULTIPOLYGON (((127.0087950432604 37.184159165...,0,1,0,1,0.0,0,0,...,0,0,0,1,0,42.714375,4779.710000,0.232100,4.630,0.50
1,다사569091,0,MULTIPOLYGON (((127.0144567913441 37.179675470...,0,0,0,0,0.0,0,0,...,0,0,0,1,0,17.366065,2982.066667,0.338792,35.005,0.50
2,다사569092,0,MULTIPOLYGON (((127.0144510220025 37.180576853...,0,1,0,0,0.0,0,0,...,0,0,1,0,1,27.409931,4835.990000,0.232096,5.285,0.50
3,다사569093,0,MULTIPOLYGON (((127.014445252404 37.1814782360...,0,0,0,0,0.0,0,0,...,0,0,0,0,1,7.110000,4835.990000,0.232096,5.285,0.50
4,다사570088,0,MULTIPOLYGON (((127.0156005712868 37.176975930...,0,0,0,0,0.0,0,0,...,0,0,3,0,0,11.820111,1607.680000,0.372530,99.780,0.25


In [7]:
from geoband.API import *
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import random
from sklearn import neighbors

In [8]:
dataset = dataset.drop(columns=['gid','geometry'])

In [9]:
dataset1 = dataset[dataset['accident_cnt']>0]
dataset1 = dataset1.reset_index(drop=True)
dataset1['for_split'] = 1
dataset2 = dataset[dataset['accident_cnt']==0]
dataset2 = dataset2.reset_index(drop=True)
dataset2['for_split'] = 0

In [10]:
X1 = dataset1.drop(['accident_cnt'],axis=1)
y1 = dataset1.accident_cnt
random.seed(210124)
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.20, random_state=4)
X2 = dataset2.drop(['accident_cnt'],axis=1)
y2 = dataset2.accident_cnt
random.seed(210124)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.20, random_state=4)

X_train = pd.concat([X1_train,X2_train])
X_train = X_train.reset_index()
X_train = X_train.drop(['index'],axis=1)
X_test = pd.concat([X1_test,X2_test])
X_test = X_test.reset_index()
X_test = X_test.drop(['index','for_split'],axis=1)
y_train = pd.concat([y1_train,y2_train])
y_train = y_train.reset_index()
y_train = y_train.drop(['index'],axis=1)
y_test = pd.concat([y1_test,y2_test])
y_test = y_test.reset_index()
y_test = y_test.drop(['index'],axis=1)

In [11]:
X_train = pd.concat([X1_train,X2_train])
X_train = X_train.reset_index()
X_train = X_train.drop(['index'],axis=1)
X_test = pd.concat([X1_test,X2_test])
X_test = X_test.reset_index()
X_test = X_test.drop(['index','for_split'],axis=1)
y_train = pd.concat([y1_train,y2_train])
y_train = y_train.reset_index()
y_train = y_train.drop(['index'],axis=1)
y_test = pd.concat([y1_test,y2_test])
y_test = y_test.reset_index()
y_test = y_test.drop(['index'],axis=1)

In [17]:
from sklearn.model_selection import StratifiedKFold
random.seed(210124)
skf = StratifiedKFold(n_splits=5,shuffle = True)
X = X_train.drop(['for_split'],axis=1)
y = y_train
target = X_train.for_split
prediction = np.zeros(len(X_test))
avg_mse = 0
avg_rmse = 0
avg_msle = 0
avg_rmsle = 0
avg_r2 = 0

In [18]:
avg_mse_ls = []
avg_rmse_ls = []
avg_msle_ls = []
avg_rmsle_ls = []
avg_y_pred = []
y_pred_df = pd.DataFrame()

random.seed(210124)

for i in range(3,20):
    n_neighbors = i
    print("n_neighbors: ", n_neighbors)
    for fold_id, (train_idx,val_idx) in enumerate(skf.split(X,target)):
        print(train_idx,val_idx)
        X_train_cv, X_val = X.iloc[train_idx,], X.iloc[val_idx,]
        y_train_cv, y_val = y.iloc[train_idx,], y.iloc[val_idx,]

        sc = StandardScaler()
        X_train_cv = sc.fit_transform(X_train_cv)
        X_val = sc.transform(X_val)
        X_testt = sc.fit_transform(X_test)
        X_train_cv = pd.DataFrame(X_train_cv, columns = X_test.columns)
        X_val = pd.DataFrame(X_val, columns = X_test.columns)
        X_testt = pd.DataFrame(X_test, columns = X_test.columns)
        
        knn = neighbors.KNeighborsRegressor(n_neighbors)
        knnfit = knn.fit(X_train_cv, y_train_cv)

        y_pred = knnfit.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
        rmse = np.sqrt(mse)
        msle = mean_squared_log_error(y_val, y_pred + 1)
        rmsle = np.sqrt(msle)
        
        print(y_pred.shape)
        
        #print(y_pred)
        #print('Mean Squared Error :', mse)
        #print('Root Mean Squared Error :', rmse)
        #print('Mean Squared Log Error :', msle)
        #print('Root Mean Squared Log Error :', rmsle)
        
        y_pred_df = pd.concat([y_pred_df,pd.DataFrame(y_pred)],axis =1)
    #y_pred_avg = y_pred_df.mean(axis=1)
    avg_mse += mse/skf.n_splits
    avg_rmse += rmse/skf.n_splits
    avg_msle += msle/skf.n_splits
    avg_rmsle += rmsle/skf.n_splits
    
    #avg_y_pred.append(list(y_pred_avg))    
    avg_mse_ls.append(avg_mse)
    avg_rmse_ls.append(avg_rmse)
    avg_msle_ls.append(avg_msle)
    avg_rmsle_ls.append(avg_rmsle)         

n_neighbors:  3
[   0    2    3 ... 1270 1271 1272] [   1    6   13   14   21   23   24   26   29   31   33   34   37   42
   45   51   55   57   60   63   64   70   79   81   82   83   91   96
   99  109  112  121  127  145  151  164  173  174  175  185  187  189
  199  219  228  233  237  238  239  245  250  255  259  264  267  270
  271  273  274  285  293  295  300  306  308  310  320  337  340  341
  342  347  350  371  379  385  388  390  392  399  404  405  410  411
  420  428  439  449  450  464  468  469  473  505  506  520  524  525
  538  539  545  549  552  557  562  563  569  572  576  583  586  591
  600  604  605  606  609  621  623  625  627  631  638  641  643  647
  649  651  653  655  661  667  671  673  674  676  686  702  709  710
  716  720  721  722  727  751  755  764  765  770  774  779  784  785
  789  795  800  804  805  806  807  808  810  818  820  821  822  828
  830  831  832  835  840  841  845  849  850  851  855  858  859  860
  910  913  914  920  935

In [19]:
pd.concat([pd.DataFrame({"n": list(range(3,20))}),pd.DataFrame({"avg_mse":avg_mse_ls}),
           pd.DataFrame({"avg_rmse":avg_rmse_ls}),pd.DataFrame({"avg_msle":avg_msle_ls}),pd.DataFrame({"avg_rmsle:":avg_rmsle_ls})],axis = 1)

,n,avg_mse,avg_rmse,avg_msle,avg_rmsle:
0,3,0.121260,0.155730,0.103506,0.143879
1,4,0.270620,0.328566,0.207115,0.287830
2,5,0.397864,0.488093,0.313197,0.433488
3,6,0.499133,0.630408,0.414913,0.576118
4,7,0.648369,0.803172,0.518128,0.719795
5,8,0.774920,0.962263,0.622565,0.864319
6,9,0.876174,1.104569,0.726294,1.008353
7,10,0.961796,1.235429,0.828159,1.151087
8,11,1.038304,1.359129,0.932645,1.295646
9,12,1.128587,1.493504,1.035749,1.439245


네이버후드 3이 제일 좋다

In [20]:
X_trainn = X_train.drop(columns = ["for_split"])

In [21]:
knnfinal = neighbors.KNeighborsRegressor(3)
knnfinal = knnfinal.fit(X_trainn, y_train)

y_test_pred = knnfinal.predict(X_test)

In [22]:
mse = mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(mse)
msle = mean_squared_log_error(y_test, y_test_pred + 1)
rmsle = np.sqrt(msle)

In [23]:
print("mse: ",mse)
print("rmse: ",rmse)
print("msle: ",msle)
print("rmsle: ",rmsle)

mse:  0.7241379310344828
rmse:  0.8509629433967631
msle:  0.5388286859054058
rmsle:  0.7340495118896312
